# Widzenie komputerowe - projekt 2
## Temat: Śledzenie gry w bilard
Autorzy:
- Michał Ciesielski 145325
- Mateusz Frąckowiak 145264
- Paweł Chumski 144392

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import math

from PIL import Image
from pprint import pprint
from ipywidgets import Video


from google.colab import drive

In [ ]:
def imshow(a):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(a))

## Wykrywanie stołu
Początkowo wykrywany jest obszar zielony reprezentujący pole do gry. W tym celu na obrazie źródłowym wykonywane jest rozmycie Gaussowskie, transformacja do przestrzeni barw HSV oraz operacja zamkniecia morfologicznego. Następnie na podstawie wykrytego obszaru wykonywana jest transformata Hough'a w celu znalezienia linii prezentujących granicę stołu, która będzie odporna na różnego typu zakłócenia (opierający się snookerzysta, przechodzący sędzia). Duplikaty znalezionych tych samych linii są usuwane. Na podstawie wykrytych linii znajdowane są łuzy, które znajdują się na przecięciach głównych linii ograniczających oraz na środku stołu (średnia wyznaczona na podstawie wyznaczonych pozostałych punktów).

### Funkcja służąca do wykrycia zielonego obszaru stołu (obszaru gry)

In [ ]:
LOWER_THRESHOLD = np.array([30, 210,80]) 
UPPER_THRESHOLD = np.array([85, 255,200])

def find_table_mask(frame):
    blurred = cv2.GaussianBlur(frame,(0,0), 2)

    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, LOWER_THRESHOLD, UPPER_THRESHOLD)

    kernel = np.ones((10,10), np.uint8)
    mask_closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    return mask_closing

### Funkcja znajdująca zduplikowane linie (przeznaczone do usunięcia). Wartości opisujące proste są odpowiednio przeskalowane w celu znalezienia podobieństwa

In [ ]:
def find_lines_to_delete(lines):
    to_del = np.zeros(len(lines))

    scaled = lines / 100

    for i in range(len(scaled)):
        found = False
        for j in range(0, i):
            if np.allclose(scaled[i], scaled[j], atol=0.2):
                found = True

        if found:
            to_del[i] = 1

    return to_del

### Funkcja zamieniająca proste zwracane przez transformatę Hough'a (we współrzędnych biegunowych) na dwa punkty je definiujące (w układzie kartezjańskim)




In [ ]:
def get_line_points(lines):
    points = []
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 2000*(-b)), int(y0 + 2000*(a)))
        pt2 = (int(x0 - 2000*(-b)), int(y0 - 2000*(a)))
        points.append((pt1, pt2))
    return points

### Funkcje liczące współczynnik kierunkowy oraz tangens kąta pomiędzy dwoma prostymi

In [ ]:
def calc_slope(line):
    p1, p2 = line
    return (p2[1] - p1[1]) / (p2[0] - p1[0])

def calc_tangent_between_lines(line1, line2):
    slope1 = calc_slope(line1)
    slope2 = calc_slope(line2)
    return np.abs((slope2 - slope1) / (1 + slope1 * slope2))

Funkcja znajdująca punkty przecięcia się prostych - łuzy znajdujące się w narożnikach stołu

In [ ]:
def find_corners(lines):
    perpendicular = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            tangent = calc_tangent_between_lines(lines[i], lines[j])
            if tangent > 3:
                perpendicular.append((lines[i], lines[j]))

    corners = []
    for l1, l2 in perpendicular:
        a1 = calc_slope(l1)
        a2 = calc_slope(l2)
        b1 = l1[0][1] - a1 * l1[0][0]
        b2 = l2[0][1] - a2 * l2[0][0]

        x = (b2 - b1) / (a1 - a2)
        y = a1 * x + b1

        corners.append((int(x), int(y)))
    return corners

Funkcja znajdująca łuzy znajdujące się w środkowej części dłuższego boku stołu na podstawie wyznaczonych narożników

In [ ]:
def find_middle_pocket_coords(corners):
    middle_pocket_coords = []
    for i in range(len(corners)):
        for j in range(i+1, len(corners)):
            p1 = corners[i]
            p2 = corners[j]
            if math.isclose(p1[0], p2[0], rel_tol=0.4):
                mid_point = (int((p1[0] + p2[0])/2), int((p1[1] + p2[1])/2.37))
                middle_pocket_coords.append(mid_point)
    return middle_pocket_coords

## Funkcje wykrywające położenie łuz
Wykorzystując powyższe funkcje, znajdowane i zwracane są wszystkie łuzy

In [ ]:
def get_pockets(frame):
    table_mask = find_table_mask(frame)

    edges = cv2.Canny(table_mask,80,200)

    all_lines = cv2.HoughLines(edges, 1, np.pi / 180, 100, None, 0, 0)

    to_del = find_lines_to_delete(all_lines)

    lines = [l for i, l in enumerate(all_lines) if not to_del[i]]

    if len(lines) != 4:
        return []

    line_points = get_line_points(lines)

    if len(line_points) != 4:
        return []

    corners = find_corners(line_points)

    if len(corners) != 4:
        return []

    middle_pocket_coords = find_middle_pocket_coords(corners)

    if len(middle_pocket_coords) != 2:
        return []

    return corners + middle_pocket_coords


## Funkcja wykrywająca bile
Poniższa funkcja wykrywa pozycje bil na podstawie przygotowanych wzorców. Po wykonaniu funkcji dopasowującej wzorzec do obrazu brany jest punkt o największym podobieństwie do podanego wzorca. Jeśli podobieństwo jest wystarczająco wysokie, pozycja bili jest zaznaczana na obrazie i zwracana do dalszego śledzenia. Z uwagi na duże skupienie bil czerwonych wokół siebie i ich dużą liczbę, powyższy proces powtarzany jest kilkukrotnie w celu uzyskania lepszych rezultatów. W celu uniknięcia problemu zaznaczania jednej bili wielokrotnie, dopasowywanie wzorców dokonywane jest na kopii obrazu.

In [ ]:
def detect_bills(file_template, img, img_copy, thresh):
    result = []
    template = cv2.imread(f'/content/drive/MyDrive/wk_projekt_2/templates/{file_template}.png', cv2.IMREAD_COLOR)
    _, w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_copy, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left_corner = max_loc
    bottom_right_corner = (top_left_corner[0] + w, top_left_corner[1] + h)
    colors = dict(pink=(153, 51, 255), blue=(255, 0, 0), red=(0, 0, 255),
                red2=(0, 0, 255), red3=(0, 0, 255), black=(0, 0, 0),
                brown=(0, 51, 102), green=(0, 255, 0), white=(255, 255, 255),
                yellow=(0, 255, 255))
    pomT = 0.83
    if file_template in ('red', 'red2', 'red3'):
        pomT += 0.07

    if max_val > pomT:
        cv2.rectangle(img, top_left_corner, bottom_right_corner, colors[file_template], 2)
        cv2.rectangle(img_copy, top_left_corner, bottom_right_corner, (0, 0, 0), -1)
        result.append((colors[file_template], (*top_left_corner, w, h)))
  
    if file_template in ('red', 'red2', 'red3'):
        while max_val > thresh:
            res = cv2.matchTemplate(img_copy, template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            top_left_corner = max_loc
            bottom_right_corner = (top_left_corner[0] + w, top_left_corner[1] + h)

            cv2.rectangle(img, top_left_corner, bottom_right_corner, colors[file_template], 2)
            cv2.rectangle(img_copy, top_left_corner, bottom_right_corner, (0,0,0), -1)
            result.append((colors[file_template], (*top_left_corner, w, h)))

    return result

## Funkcja załadowująca wideo oraz obrysowująca kontury znalezionej bili

In [ ]:
def load_video(path):
    video = cv2.VideoCapture(path)
    if video.isOpened():
        print('Film wczytany!')

    video_width = int(video.get(3))
    video_height = int(video.get(4))

    print(video_height, video_width)

    video_fps = video.get(cv2.CAP_PROP_FPS)
    print(video_fps)

    return video, (video_width, video_height), video_fps

def draw_bbox(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

## Główna funkcja przetwarzająca wideo
Po załadowaniu wideo na pierwszej klatce wykrywany jest stół oraz bile. Dla każdej bili tworzony jest tracker. Pozycje bil są aktualizowane poprzez wskazania ich trackerów oraz sprawdzane są zdarzenia uderzenia bili przez bile białą (czy dana bila zmieniła pozycję), wbicia bili do łuzy (czy zmieniła pozycję i była na tyle blisko łuzy aby wpaść) oraz faulu (biała bila poruszyła się ale się zatrzymała i nie uderzyła w inną bilę). Może się zdarzyć sytuacja, że faul zostanie wykryty przed zatrzymaniem się bili ale po zderzeniu z inną bilą informacja o faulu zniknie.

### Zamontowanie dysku z plikami

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### Przetwarzanie

In [ ]:
video, video_shape, video_fps = load_video('/content/drive/MyDrive/wk_projekt_2/hit.mp4')
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

optical_flow = cv2.VideoWriter('./result.avi', cv2.VideoWriter_fourcc(*'DIVX'), video_fps, video_shape)
pockets = None

positions = []
is_hit = []
hit = 0
balls_in_pocket = 0
message = ""
last_white_position = []
white_stop = False
is_in_pocket = []


while video.isOpened():
    ret, frame = video.read()

    if ret:
        if pockets is None:
            template_name_bills = ['white', 'blue', 'black', 'brown', 'green', 'yellow', 'red', 'red2', 'red3', 'pink']
            test = frame.copy()
            test2 = frame.copy()
            trackers = []

            for n in template_name_bills:
                bboxes = detect_bills(n, test, test2, 0.83)
                for color, bbox in bboxes:
                    positions.append(np.array(bbox[0:2]))
                    is_hit.append(False)
                    is_in_pocket.append(False)
                    if n == "white":
                        last_white_position = np.array(bbox[0:2])
                    tracker = cv2.TrackerCSRT_create()
                    tracker.init(frame, bbox)
                    trackers.append((color, tracker))

            pockets = get_pockets(frame)
            if len(pockets) != 6:
                pockets = None

      
        for i, (color, tracker) in enumerate(trackers):
            ok, bbox = tracker.update(frame)
            if ok: 
                draw_bbox(frame, bbox, color)
                if i == 0:
                    if cv2.norm(np.array(bbox)[0:2] - last_white_position, cv2.NORM_L2) < 5 and is_hit[i] == True:
                        white_stop = True
                    last_white_position = np.array(bbox[0:2])
                if cv2.norm(np.array(bbox)[0:2] - positions[i], cv2.NORM_L2) > 10 and is_hit[i] == False:
                    is_hit[i] = True
                    if i != 0:
                        hit += 1
                if i != 0 and is_hit[i] == True and is_in_pocket[i] == False:
                    for pocket in pockets:
                        if cv2.norm(pocket - np.array(bbox)[0:2], cv2.NORM_L2) < 20:
                            is_in_pocket[i] = True
                            balls_in_pocket += 1
                
        cv2.putText(frame, f"ball hit: {hit}", (15, 40), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
        cv2.putText(frame, f"in pocket: {balls_in_pocket}", (15, 80), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255), 2)
        if white_stop and hit == 0:
            cv2.putText(frame, f"foul", (15, 120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

        for i in pockets:
            cv2.circle(frame, (i[0], i[1]), 20, (0, 255, 0), 2)
            cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)
        #imshow(frame)
        optical_flow.write(frame)

    else:
        break

Film wczytany!
720 1280
25.0


In [ ]:
video.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i result.avi -y result.mp4

[mpeg4 @ 0x56385b2b6cc0] ac-tex damaged at 22 2
[mpeg4 @ 0x56385b2b6cc0] Error at MB: 184


In [ ]:
Video.from_file('./result.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x08N\xbemdat\x00\x…

## Krótki opis uzyskanych rezultatów
Zbiór testowanych filmików oraz rezultaty znajdują się pod następującym linkiem: https://drive.google.com/drive/folders/1WFLMi9YEF3K59WheBAkSF-Q7jztaRHYT?fbclid=IwAR3iM2_x8Ii-aIRrFoo7hDoVXLaehwndA1sp53_7vLCQe4MIRlhyyKcAmM0. 

Dużą trudnością okazało się znalezienie odpowiednich fragmentów ukazujących  cały stół z rozgrywek w Snookera (poszczególne transmisje często ukazują ujęcia 
z różnych kamer). Problemem mogą być również różne ujęcia kamer, stopień naświetlenia sceny, jakość nagrania itp. Niemniej jednak dla nagrań z Mistrzostw Świata w Snookerze z 2022 roku (przykłady pochodzą z finału), uzyskane rozwiązanie działa w pełni poprawnie.